In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tnrange, tqdm_notebook
import gc

In [2]:
sns.set_context('talk')

In [3]:
import skimage.io as io

In [ ]:
DEBUG = True

In [ ]:
import theano
theano.config.openmp = True

In [ ]:
import keras

# Preprocessing

In [6]:
labels = pd.read_csv('finetuning-train/gt.csv')

In [7]:
labels.head()

,filename,class_id
0,0000.png,50
1,0001.png,24
2,0002.png,25
3,0003.png,37
4,0004.png,13


In [8]:
labels.max()

filename    2499.png
class_id          50
dtype: object

In [9]:
labels.min()

filename    0000.png
class_id           1
dtype: object

Растянем/сожмём изображения, пока большая из сторон не станет равна SIDE_SIZE

In [10]:
SIDE_SIZE = 500

и дополним изображение до квадрата, заполняя отсутствующие клетки CVAL

In [11]:
CVAL = 255 // 2

In [13]:
from os import listdir
from os.path import isfile, join
import warnings
import skimage.color
import skimage.transform

In [54]:
def resize(input_dir, output_dir):
    files = sorted([f for f in listdir(input_dir) if isfile(join(input_dir, f))])
    for file in files:
        img = io.imread(join(input_dir, file))
        if img.ndim == 2:
            img = skimage.color.gray2rgb(img)
        factor = min([SIDE_SIZE / img.shape[i] for i in range(2)])
        resized_img = skimage.transform.rescale(img, factor,
                                               mode='constant', cval=CVAL, preserve_range=True)
        square_image = np.zeros((SIDE_SIZE, SIDE_SIZE, 3), dtype=np.float64) + CVAL
        square_image[(SIDE_SIZE - resized_img.shape[0]) // 2 :
                     resized_img.shape[0] + (SIDE_SIZE - resized_img.shape[0]) // 2,
                    (SIDE_SIZE - resized_img.shape[1]) // 2 :
                     resized_img.shape[1] + (SIDE_SIZE - resized_img.shape[1]) // 2] = resized_img
        resized_img = square_image.astype(np.uint8)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            io.imsave(join(output_dir, file), resized_img)

In [56]:
%%time
resize('finetuning-train/images/', 'finetuning-train/resized/')

CPU times: user 7min 4s, sys: 7min 56s, total: 15min 1s
Wall time: 8min 6s


In [57]:
%%time
resize('finetuning-test/images/', 'finetuning-test/resized/')

CPU times: user 1min 27s, sys: 1min 36s, total: 3min 3s
Wall time: 1min 36s


Раскидаем train по папочкам-классам

In [58]:
import os

In [69]:
def make_dirs(work_dir):
    for index in range(1, 51):
        os.mkdir(join(work_dir, "{:02d}".format(index)))

In [74]:
make_dirs('finetuning-train/resized')

In [75]:
import shutil

In [79]:
for row in labels.itertuples():
    shutil.move(join('finetuning-train/resized', row.filename), 
                join('finetuning-train/resized', "{:02d}".format(row.class_id), row.filename))

Теперь организуем такую же по структуре папку validation, и переместим туда по 10 случайных фотографий из каждого класса

In [80]:
make_dirs('finetuning-train/validation')

In [81]:
import random

def make_validation(input_dir, output_dir):
    files = random.sample([f for f in listdir(input_dir) if isfile(join(input_dir, f))], 10)
    for file in files:
        shutil.move(join(input_dir, file), join(output_dir, file))

In [82]:
for index in range(1, 51):
    make_validation(join('finetuning-train/resized', "{:02d}".format(index)),
                    join('finetuning-train/validation', "{:02d}".format(index)))

Нам понадобятся DataGenerator'ы для обучения и валидации.

In [83]:
from keras.preprocessing.image import ImageDataGenerator

In [84]:
data_gen_args = dict(rotation_range=40.,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2,
                     fill_mode='constant',
                     cval=CVAL,
                     horizontal_flip=True,
                     vertical_flip=False,)

Напишем обёртку, которая будет вызывать prepocess_input для картинок из генератора

In [85]:
from keras.applications.inception_v3 import preprocess_input

def preprocessed_datagen(datagen):
    for x, y in datagen:
        yield preprocess_input(x), y

In [120]:
batch_size=10

train_gen = preprocessed_datagen(ImageDataGenerator(**data_gen_args).flow_from_directory(
            'finetuning-train/resized', target_size=(SIDE_SIZE, SIDE_SIZE), class_mode='categorical',
             batch_size=batch_size, shuffle=True, seed=10))

validation_gen = preprocessed_datagen(ImageDataGenerator(**data_gen_args).flow_from_directory(
                 'finetuning-train/validation', target_size=(SIDE_SIZE, SIDE_SIZE), class_mode='categorical',
                 batch_size=batch_size, shuffle=True, seed=10))

Found 2000 images belonging to 50 classes.
Found 500 images belonging to 50 classes.


# Model

Будем использовать inceptionV3. Для начала снимем полносвязные слои

In [105]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense
from keras.layers import Input

base_model = InceptionV3(weights='imagenet', include_top=False, 
                         input_tensor=Input(shape=(SIDE_SIZE, SIDE_SIZE, 3)))

Добавим два полносвязных слоя

In [123]:
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.layers import Dropout
from keras.layers import Flatten, GlobalAveragePooling2D

In [108]:
lr=5 * 10 ** -3
decay=10 ** -7
w_reg=5 * 10 ** -5

In [124]:
gc.collect()

9

In [131]:
x = base_model.output

x = GlobalAveragePooling2D()(x)
x = Dense(200, activation='relu', W_regularizer=l2(w_reg))(x)
x = Dropout(0.1)(x)
x = Dense(200, activation='relu', W_regularizer=l2(w_reg))(x)
x = Dropout(0.1)(x)

predictions = Dense(50, activation='softmax', W_regularizer=l2(w_reg))(x)

model = Model(input=base_model.input, output=predictions)

for layer in base_model.layers:
    layer.trainable = False
    
model.compile(#optimizer=SGD(lr=lr, decay=decay, nesterov=True, momentum=0.9),
              optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy', 'categorical_crossentropy'])

# Learning

In [132]:
%%time

hist = model.fit_generator(
        train_gen,
        samples_per_epoch=5000,
        nb_epoch=50,
        validation_data=validation_gen,
        nb_val_samples=1000,
        )

KeyboardInterrupt: 